# Introduction
A very important aspect of supervised and semi-supervised machine learning is the quality of the labels produced by human labelers. Unfortunately, humans are not perfect and in some cases may even maliciously label things incorrectly. In this assignment, you will evaluate the impact of incorrect labels on a number of different classifiers.

We have provided a number of code snippets you can use during this assignment. Feel free to modify them or replace them.


## Dataset
The dataset you will be using is the [Adult Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult). This dataset was created by Ronny Kohavi and Barry Becker and was used to predict whether a person's income is more/less than 50k USD based on census data.

### Data preprocessing
Start by loading and preprocessing the data. Remove NaN values, convert strings to categorical variables and encode the target variable (the string <=50K, >50K in column index 14).

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder


In [2]:
# This can be used to load the dataset
data = pd.read_csv("adult.csv", na_values='?')

print('Number of samples before removing nan-values: ' + str(data.shape[0]))

data.dropna(inplace=True)

print('Number of samples after removing nan-values: ' + str(data.shape[0]))

data['salary'] = data['salary'].apply(lambda x: x.strip('.')) # Remove trailing dot

print(data.groupby('salary').size())

data.head()

Number of samples before removing nan-values: 48842
Number of samples after removing nan-values: 45222
salary
<=50K    34014
>50K     11208
dtype: int64


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Data classification
Choose at least 4 different classifiers and evaluate their performance in predicting the target variable. 

#### Preprocessing
Think about how you are going to encode the categorical variables, normalization, whether you want to use all of the features, feature dimensionality reduction, etc. Justify your choices 

A good method to apply preprocessing steps is using a Pipeline. Read more about this [here](https://machinelearningmastery.com/columntransformer-for-numerical-and-categorical-data/) and [here](https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf). 

<!-- #### Data visualization
Calculate the correlation between different features, including the target variable. Visualize the correlations in a heatmap. A good example of how to do this can be found [here](https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec). 

Select a features you think will be an important predictor of the target variable and one which is not important. Explain your answers. -->

#### Evaluation
Use a validation technique from the previous lecture to evaluate the performance of the model. Explain and justify which metrics you used to compare the different models. 

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, OrdinalEncoder, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [101]:
cats = ['workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'native-country']
nums = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']

# cats = ['occupation']
# nums = ['hours-per-week']

# We have not validated that the rest of the data is of sufficient quality.
# We only found that salary has trailing dots some times.

steps = [
    ('cat', OneHotEncoder(), cats), # Maybe try one hot encoding.
    ('num', StandardScaler(), nums)      # Other normalizing for the numericals
]

# Combine steps into a ColumnTransformer
ct = ColumnTransformer(steps)

le = LabelEncoder()
y = le.fit_transform(data['salary']).astype(int)

# show the correlation between different features including target variable
def visualize(data, ct):
    X = ct.fit_transform(data)
    X = pd.DataFrame(X)
    X['salary'] = y

    plt.matshow(X.corr())
    plt.colorbar()
    plt.show()

# Apply your model to feature array X and labels y
def apply_model(model, X, y):    
    # Wrap the model and steps into a Pipeline
    pipeline = Pipeline(steps=[('t', ct), ('m', model)])
    # Evaluate the model and store results
    return evaluate_model(X, y, pipeline)

# Apply your validation techniques and calculate metrics
def evaluate_model(X, y, pipeline):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    # pipeline.fit(X_train, y_train)      # Fits the model using the transformed data
    # s = pipeline.score(X_test, y_test)  # Uses the standard scoring function of the model

    cv = cross_val_score(pipeline, X, y, cv=3, scoring='f1', n_jobs=-1)

    return np.nanmean(cv)

# visualize(data, ct)

# These are all untuned, can do some work by setting the parameters for each model.
# Tree 
# print("Tree score:", apply_model(DecisionTreeClassifier(), data, y))

# Random forest
# print("Random Forest score:", apply_model(RandomForestClassifier(), data, y))

# # SVC
# print("SVC score:", apply_model(SVC(), data, y))

# # K - nearest neighbours
print("K nearest score:", apply_model(KNeighborsClassifier(), data, y))

K nearest score: 0.6318908441269446


In [98]:
# TODO: Check that all these are valid and/or needed:
for cats in ct.transformers_[0][1].categories_:
    print(cats)

['Federal-gov' 'Local-gov' 'Private' 'Self-emp-inc' 'Self-emp-not-inc'
 'State-gov' 'Without-pay']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
['Female' 'Male']
['Cambodia' 'Canada' 'China' 'Columbia' 'Cuba' 'Dominican-Republic'
 'Ecuador' 'El-Salvador' 'England' 'France' 'Germany' 'Greece' 'Guatemala'
 'Haiti' 'Holand-Netherlands' 'Honduras' 'Hong' 'Hungary' 'India' 'Iran'
 'Ireland' 'Italy' 'Jamaica' 'Japan' 'Laos' 'Mexico' 'Nicaragua'
 'Outlying-US(Guam-USVI-etc)' 'Peru' 'Philippines'

### Label perturbation
To evaluate the impact of faulty labels in a dataset, we will introduce some errors in the labels of our data.


#### Preparation
Start by creating a method which alters a dataset by selecting a percentage of rows randomly and swaps labels from a 0->1 and 1->0. 


In [ ]:
"""Given a label vector, create a new copy where a random fraction of the labels have been flipped."""
def pertubate(y: np.ndarray, fraction: float) -> np.ndarray:
    copy = data.copy()
    # Flip fraction*len(data) of the labels in copy
    return copy

#### Analysis
Create a number of new datasets with perturbed labels, for fractions ranging from `0` to `0.5` in increments of `0.1`.

Perform the same experiment you did before, which compared the performances of different models except with the new datasets. Repeat your experiment at least 5x for each model and perturbation level and calculate the mean and variance of the scores. Visualize the change in score for different perturbation levels for all of the models in a single plot. 

State your observations. Is there a change in the performance of the models? Are there some classifiers which are impacted more/less than other classifiers and why is this the case?

In [ ]:
# Code

Observations + explanations: max. 400 words

#### Discussion

1)  Discuss how you could reduce the impact of wrongly labeled data or correct wrong labels. <br />
    max. 400 words



    Authors: Youri Arkesteijn, Tim van der Horst and Kevin Chong.


## Machine Learning Workflow

From part 1, you will have gone through the entire machine learning workflow which are they following steps:

1) Data Loading
2) Data Pre-processing
3) Machine Learning Model Training
4) Machine Learning Model Testing

You can see these tasks are very sequential, and need to be done in a serial fashion. 

As a small perturbation in the actions performed in each of the steps may have a detrimental knock-on effect in the task that comes afterwards.

In the final part of Part 1, you will have experienced the effects of performing perturbations to the machine learning model training aspect and the reaction of the machine learning model testing section.

## Part 2 Data Discovery

Now we have a some datasets that are related to each other.

**Altogether they are the same as the adult dataset used in the part 1 of the assignment.**

In this scenario, one can see the utility of the subsets of data can impact the outcome of the task from the previous section.

Because the data is split up, we want to be able to re-construct the data through data discovery.

As data discovery will allow you to be able to find relations that can be used to reconstruct the entire dataset.

Implement a method of your choice to perform the data discovery to be able to recover the entire dataset from part 1 of the assignment.

In [5]:
def discovery_algorithm(datasets: list, threshold: float) -> list:
    """Function should be able to perform data discovery to find related datasets
    Possible Input: List of datasets
    Output: List of pairs of related datasets
    """
    similar_datasets = []
    for i, dataset1 in enumerate(datasets):
        for j, dataset2 in enumerate(datasets):
            if i == j or i > j:
                continue
            col1 = dataset1.columns.str.lower()
            col2 = dataset2.columns.str.lower()
            #calculate intersection
            intersection = col1.intersection(col2)
            union = col1.union(col2)
            #calculate similarity
            similarity = len(intersection)/len(union)
            # if similarity >= threshold:
            similar_datasets.append((i, j, similarity, intersection))

    similar_datasets.sort(key=lambda x: x[2], reverse=True)
    return similar_datasets

def merge_columns(merged_dataset: pd.DataFrame, intersection: list) -> pd.DataFrame:
    for column in intersection:
        merged_dataset[column] = merged_dataset.apply(lambda row: merge_column(row[f"{column}_df1"], row[f"{column}_df2"], column), axis=1)

    merged_dataset.drop(columns=[f"{col}_df1" for col in intersection] + [f"{col}_df2" for col in intersection], inplace=True)
    return merged_dataset

def merge_column(val1, val2, column):
    if pd.isna(val1):
        return val2
    elif pd.isna(val2):
        return val1
    elif val1 == val2:
        return val1
    else:
        return None  # or some default value	

In [22]:
## Do some testing using different degrees of relations on the downstream task.
##

def read_data(file_name):
    df = pd.read_csv(file_name, na_values='?').iloc[: , 1:]
    if 'salary' in df.columns:
        df['salary'] = df['salary'].str.replace('.', '')
    return df

dataset_names = ["adult_split/data_adult_0_1.csv", "adult_split/data_adult_0_2.csv", "adult_split/data_adult_1_1.csv", "adult_split/data_adult_1_2.csv", "adult_split/data_adult_2_1.csv", "adult_split/data_adult_2_2.csv"]

select_columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","salary"]
datasets = [read_data(name) for name in dataset_names]


#merge similar datasets
while len(datasets) > 1:
    similar_datasets = discovery_algorithm(datasets, 0.5)
    di, dj, similarity, intersection = similar_datasets.pop(0)
    join_on  = [col for col in intersection if col in ['first_name', 'last_names', 'race', 'sex']]
    merged_dataset = pd.merge(datasets[di], datasets[dj], how='outer', on=['first_name', 'last_names'], suffixes=('_df1', '_df2'))
    merged_dataset = merge_columns(merged_dataset, list(set(intersection) - set(['first_name', 'last_names'])))
    remove_indices = [di,dj]
    datasets = [i for j, i in enumerate(datasets) if j not in remove_indices]
    datasets.append(merged_dataset)

    take_columns = [col for col in select_columns if col in merged_dataset.columns.str.lower()]
    final_dataset = merged_dataset[take_columns]
    final_dataset.to_csv(f"adult_merged/adult_merged_{5-len(datasets)}.csv", index=False)



#check overlap with actual data
pd.merge(final_dataset, data, how='inner', on=select_columns)
    


KeyError: 'workclass'

In [21]:
datasets_random = [read_data(name) for name in dataset_names]

while len(datasets_random) > 1:
    if len(datasets_random) == 6:
        di = np.random.choice([1,2,5])
        dj = np.random.choice([j for j in range(len(datasets_random)) if j != di])
    else:
        di, dj = np.random.choice(range(len(datasets_random)), 2, replace=False)
    col1 = datasets_random[di].columns.str.lower()
    col2 = datasets_random[dj].columns.str.lower()
    #calculate intersection
    intersection = col1.intersection(col2)
    join_on  = [col for col in intersection if col in ['first_name', 'last_names', 'race', 'sex']]
    merged_dataset = pd.merge(datasets_random[di], datasets_random[dj], how='outer', on=['first_name', 'last_names'], suffixes=('_df1', '_df2'))
    merged_dataset = merge_columns(merged_dataset, list(set(intersection) - set(['first_name', 'last_names'])))
    remove_indices = [di,dj]
    datasets_random = [i for j, i in enumerate(datasets_random) if j not in remove_indices]
    datasets_random.append(merged_dataset)

    take_columns = [col for col in select_columns if col in merged_dataset.columns.str.lower()]
    final_dataset = merged_dataset[take_columns]
    final_dataset.to_csv(f"adult_merged/adult_random_{5-len(datasets_random)}.csv", index=False)

pd.merge(final_dataset, data, how='inner', on=select_columns)

KeyError: 'workclass'

### Evaluation

Following the same workflow as Part 1 of the assignment, you will need to perform the steps once again.
This means that with the difference in setting, there has now been a change in the data loading portion of the workflow. 

<!-- While performing data discovery, one can check how having more data of different relations can effect the outcome of the downstream task. -->

As you perform the act of data discovery you will be piecing the data back together one by one by finding the relation between the datasets.

As you piece the data back together, the entire dataset will be available in varying portions.

Then using the different portions of available data, fit the models that were used previously, and examine the results when testing on the appropriate data from the testing samples.

*The dataset from part 1 can be treated as the groundtruth, so you can try and random sample from that dataset to produce the testing samples that can be used in this part.* 

***

One can also evaluate on the effect of incorrectness of data discovery.

Suppose what you have done for data discovery is correct, this means the relations that are found are correct.

What if you perform a perturbation on the relations between the files? 

This would mean the dataset will be incorrectly joined, do you think there will be an impact on the outcome of the model that will be trained and then tested on the groudtruth dataset?

What would the effect be on the downstream tasks as mentioned in the machine learning workflow such as data pre-processing, machine learning model training and testing?

This can be evaluated in the same way as above.

In [33]:
def run_model(model, train_x, train_y, test_x, test_y, ct):
    cats = [c for c in ['workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'native-country'] if c in train_x.columns.str.lower()]
    nums = [c for c in ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week'] if c in train_x.columns.str.lower()]

    steps = [
        ('cat', OneHotEncoder(handle_unknown="ignore"), cats), # Maybe try one hot encoding.
        ('num', StandardScaler(), nums)      # Other normalizing for the numericals
    ]

    # Combine steps into a ColumnTransformer
    ct = ColumnTransformer(steps)

    pipeline = Pipeline(steps=[('t', ct), ('m', model)])
    pipeline.fit(train_x, train_y)      # Fits the model using the transformed data
    s = pipeline.score(test_x, test_y)  # Uses the standard scoring function of the model
    return s


In [39]:
## Do some testing on performing various degrees of incorrect data discovery.
##
le = LabelEncoder()

test_data = pd.read_csv("adult.csv", na_values='?').dropna()
test_data['salary'] = test_data['salary'].apply(lambda x: x.strip('.'))
test_data = test_data.sample(frac=0.3).reset_index(drop=True)

for i in range(2, 5):
    data = pd.read_csv(f"adult_merged/adult_merged_{i}.csv", na_values='?')
    data.dropna(inplace=True)
    data['salary'] = data['salary'].apply(lambda x: x.strip('.')) # Remove trailing dot

    train_x = data.drop(columns=['salary'])
    train_y = le.fit_transform(data['salary']).astype(int)
    test_x = test_data[train_x.columns.str.lower()]
    test_y = le.fit_transform(test_data['salary']).astype(int)

    print(f"{i} merged K nearest score:", run_model(KNeighborsClassifier(n_neighbors=min(train_x.shape[0],5)), train_x, train_y, test_x, test_y, ct))
    print(f"{i} merged Random Forest score:", run_model(RandomForestClassifier(), train_x, train_y, test_x, test_y, ct))
    print(f"{i} merged SVC score:", run_model(SVC(), train_x, train_y, test_x, test_y, ct))
    print(f"{i} merged Tree score:", run_model(DecisionTreeClassifier(), train_x, train_y, test_x, test_y, ct))

K nearest score: 0.6747254367214565
Random Forest score: 0.7408417483599912
SVC score: 0.7561730670008108
Tree score: 0.7481388663669197
K nearest score: 0.8648927544777769
Random Forest score: 0.9419178889953563
SVC score: 0.8609862165548758
Tree score: 0.9248175720498267
K nearest score: 0.8771283260853542
Random Forest score: 0.9882803862312965
SVC score: 0.8627552148595857
Tree score: 0.9845212648337879


In [40]:
for i in range(2, 5):
    data = pd.read_csv(f"adult_merged/adult_random_{i}.csv", na_values='?')
    data.dropna(inplace=True)
    data['salary'] = data['salary'].apply(lambda x: x.strip('.')) # Remove trailing dot

    train_x = data.drop(columns=['salary'])
    train_y = le.fit_transform(data['salary']).astype(int)
    test_x = test_data[train_x.columns.str.lower()]
    test_y = le.fit_transform(test_data['salary']).astype(int)

    print(f"{i} random K nearest score:", run_model(KNeighborsClassifier(n_neighbors=min(train_x.shape[0],5)), train_x, train_y, test_x, test_y, ct))
    print(f"{i} random Random Forest score:", run_model(RandomForestClassifier(), train_x, train_y, test_x, test_y, ct))
    print(f"{i} random SVC score:", run_model(SVC(), train_x, train_y, test_x, test_y, ct))
    print(f"{i} random Tree score:", run_model(DecisionTreeClassifier(), train_x, train_y, test_x, test_y, ct))

2 random K nearest score: 0.8362202402889364
2 random Random Forest score: 0.8965135991744675
2 random SVC score: 0.855679221640746
2 random Tree score: 0.8687992924006781
3 random K nearest score: 0.8363676568143289
3 random Random Forest score: 0.89725068180143
3 random SVC score: 0.8561951794796197
3 random Tree score: 0.8710105402815655
4 random K nearest score: 0.8761701186703029
4 random Random Forest score: 0.9883540944939928
4 random SVC score: 0.864008255325422
4 random Tree score: 0.9859217218250166


## Discussions

1)  Discuss the different effects of the results of the data discovery results on various downstream tasks in the machine learning workflow.
    As stated previously, this is effecting the data loading portion of the ML workflow.

2)  Discuss also what aspects need to be considered when performing data discovery and evaluating the results of data discovery.

Max. 400 words

The perturbations that performed for part 1 and 2 are all data quality issues.

3) Discuss on the effects of data quality and how you may attempt to identify and solve these issues?

<!-- For the set of considerations that you have outlined for the choice of data discovery methods, choose one and identify under this new constraint, how would you identify and resolve this problem? -->

Max 400 words